In [8]:
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
from utilities import create_inc, read_image
import tifffile
import radiometric_calibration as rad_cal
import pandas as pd
import numpy as np
import os
import config
load_dotenv()

True

In [12]:
path = r'C:\Users\Vision IAS\Desktop\work\SAR\Output\VH_calibrated.tiff'
np.abs(read_image.image_array(path))[2084,2455]

1649.5703

In [13]:
path = r'C:\Users\Vision IAS\Desktop\work\SAR\Output\HV_calibrated.tiff'
np.abs(read_image.image_array(path))[2084,2455]

1333.6559

In [14]:
path = r'C:\Users\Vision IAS\Desktop\work\SAR\Output\HH_calibrated.tiff'
np.abs(read_image.image_array(path))[2084,2455]

26633.742

In [15]:
path = r'C:\Users\Vision IAS\Desktop\work\SAR\Output\VV_calibrated.tiff'
np.abs(read_image.image_array(path))[2084,2455]

26633.742